# Capstone Project 

## Data download and cleaning

Let's start by importing BeautifulSoup to scrape the wikipedia page 

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('Libraries imported!')

Libraries imported!


Find the wikipedia page and open with BeautifulSoup

In [3]:
!wget -O filename.html en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
with open('filename.html') as html_file:
    soup= BeautifulSoup(html_file, 'lxml')

--2019-02-27 09:44:55--  http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:80... connected.
HTTP request sent, awaiting response... 301 TLS Redirect
Location: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M [following]
--2019-02-27 09:44:55--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80807 (79K) [text/html]
Saving to: ‘filename.html’

filename.html       100%[=====================>]  78.91K  --.-KB/s   in 0.05s  

2019-02-27 09:44:55 (1.52 MB/s) - ‘filename.html’ saved [80807/80807]



Parse wikipedia page for data

In [4]:
#Find the first table on page
table= soup.find('table', class_='wikitable sortable')

# Build parser
n_columns=0
n_rows=0
column_names= []

row_marker= 0 

# Find number of rows and columns
# also find columns header names 
for row in table.find_all('tr'):
    td_tags= row.find_all('td')
    if len(td_tags)>0:
        n_rows+=1
        if n_columns==0:
            n_columns== len(td_tags)
            
    #Handle column names
    th_tags= row.find_all('th')
    if len(th_tags)>0 and len(column_names)==0:
        for th in th_tags:
            column_names.append(th.get_text())

columns= column_names if len(column_names)>0 else range(0,n_columns)
df= pd.DataFrame(columns= columns, index= range(0, n_rows))
row_marker=0
for row in table.find_all('tr'):
    column_marker=0
    columns= row.find_all('td')
    for column in columns:
        df.iloc[row_marker, column_marker]= column.get_text()
        column_marker+=1
    if len(columns)>0:
        row_marker+=1

# Preview of Wikipedia postal codes as a panda DataFrame
df.head()
        
        
            

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Table shape before cleaning operations

In [5]:
df.shape

(289, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'

In [6]:
df= df.drop(df[df.Borough == 'Not assigned'].index).reset_index(drop=True)

Drop new line script(\n) from table and column header

In [7]:
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df.replace('\n', '', regex=True, inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Merge rows with same borough using commas

In [8]:
dfgrouped= df.groupby(['Postcode', 'Borough'], as_index=False, sort=False)['Neighbourhood'].apply(','.join).reset_index(name='Neighbourhood')
dfgrouped= pd.DataFrame(dfgrouped)
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Where neighbourhood name is missing, apply borough name

In [9]:
 dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned']= dfgrouped['Borough']

In [10]:
# Check if any neighbourhoods left with unassigned values
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned'].count()

0

In [11]:
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [12]:
dfgrouped.shape

(103, 3)

## Adding geospatial coordinates 

Let's add latitude and longitude coordinates for each neighbourhood

In [13]:
# open csv file 
!wget -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('data downloaded!')

--2019-02-27 09:44:56--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-27 09:44:56--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-27 09:44:57--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-27 09:44:58--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [14]:
df_geo= pd.read_csv('geospatial_data.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Shape of our geospatial data 

In [15]:
df_geo.shape

(103, 3)

Great! geospatial data from file matches the shape of our neighbourhood dataframe

Let's join the data from that file to our dataframe using post codes as index

In [16]:
df_geo= df_geo.rename(columns={'Postal Code': 'Postcode'})
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [17]:
df2= pd.merge(dfgrouped,
            df_geo[['Postcode','Latitude', 'Longitude']],
             on='Postcode')
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [18]:
# Check shape
df2.shape

(103, 5)

Our new dataframe has the same number of lines as our original dataframe

In [19]:
# Check for missing values
df2.isnull().any()

Postcode         False
Borough          False
Neighbourhood    False
Latitude         False
Longitude        False
dtype: bool

No missing data

<a id='Section2' > </a>

# Explore and cluster 

Before we start, let's import the dependencies we need for analysing json files, mapping and clustering 

In [20]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Select neighbourhoods from dataframe containing the word 'Toronto'

In [21]:
df_narrowed= df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_narrowed.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now, let's visualize this selection using geopy

Use geopy library to get the latitude and longitude values of Toronto.

In [22]:
address= 'Toronto, Ontario'
geolocator= Nominatim()
location= geolocator.geocode(address)
latitude= location.latitude
longitude= location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


Create a map of Toronto neighbourhoods with their names as labels

In [23]:
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_narrowed['Latitude'],df_narrowed['Longitude'],df_narrowed['Neighbourhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)

toronto_map

Define Foursquare credentials

In [24]:
# @hidden_cell
CLIENT_ID = '0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ' # your Foursquare ID
CLIENT_SECRET = 'OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ
CLIENT_SECRET:OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF


### Let's explore the first neighbourhood in our dataframe

Get the neighbourhood's name

In [25]:
df3= df_narrowed
df3.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

Get the neighborhood's latitude and longitude values.

In [26]:
neighbourhood_latitude= df3.loc[0, 'Latitude']
neighbourhood_longitude= df3.loc[0, 'Longitude']

neighbourhood_name= df3.loc[0, 'Neighbourhood']

print('The latitude and longitude of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

The latitude and longitude of Harbourfront,Regent Park are 43.6542599, -79.3606359.


### Now, let's get the top 100 venues that are in Harbourfront, Regent Park within a radius of 500 meters.

First let's create the GET request URL

In [27]:

LIMIT=100
radius=500
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ&client_secret=OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [28]:
results= requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c765c1ef594df76bb6bd6e1'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [29]:
# create function to find category type
def get_category_type(row):
    try:
        categories_list= row['categories']
    except:
        categories_list= row['venue.categories']
    if len(categories_list)==0:
            return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe 

In [30]:
venues= results['response']['groups'][0]['items']

nearby_venues= json_normalize(venues)

#filter columns
columns_filtered= ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues= nearby_venues.loc[:, columns_filtered]
#filter the category for each row
nearby_venues['venue.categories']= nearby_venues.apply(get_category_type, axis=1)

#clean up column titles
nearby_venues.columns= [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
8,Corktown Common,Park,43.655618,-79.356211
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


## Explore neighbourhoods in Toronto

### Let's create a function to repeat the process to all neighbourhoods in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toronto_frame= getNearbyVenues(names= df3['Neighbourhood'], latitudes= df3['Latitude'], longitudes=df3['Longitude'])

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


Let's check the size of our dataframe

In [33]:
print(toronto_frame.shape)

(1702, 7)


In [34]:
toronto_frame.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues are listed for each neighbourhood

In [35]:
toronto_frame.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",49,49,49,49,49,49
Central Bay Street,81,81,81,81,81,81
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


### Let's find out how many venue category types can be curated 

In [36]:
print('there are {} uniques categories.'.format(len(toronto_frame['Venue Category'].unique()))) 
'Neighborhood'in toronto_frame['Venue Category'].values

there are 240 uniques categories.


True

In [37]:
# Set pandas viewing option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Analyse each neighbourhood

In [38]:
# One hot encoding 
toronto_onehot= pd.get_dummies(toronto_frame[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0